# Event Building from HDF5 Files
-----
In this example notebook, we will show how to carry out random and threshold triggering on a continuous stream of data, with data acquired via the Moku.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import splendsp as sp
import splendaq as sq

We will first generate some continuous pulse data with the Moku. See the `logging_data.ipynb` tutorial for further explanation on how to log data, as this notebook is for offline triggering after the data has been logged.

The setup we are assuming here is that Output1 one of the Moku is directly connected to Input1, sending square pulses as a rate of 10 Hz for 30 s.

In [ ]:
fs = 500e3
moku_ip = 'your_moku_ip'

pulse_width = 1e-3
edge_time = 50e-6

with sq.daq.LogData(moku_ip, fs=fs, force_connect=True) as LOG:
    LOG.set_input_channels(1)
    pulse_settings = LOG.pulse_settings(
        amplitude=0.2,
        frequency=10,
        offset=0,
        edge_time=edge_time,
        pulse_width=pulse_width,
    )
    LOG.set_output_channel(1, 'Pulse', **pulse_settings)

    LOG.log_data(30)

We now have a 30 second long file of continuous data, which will we need to convert to a `splendaq` HDF5 file for compatibility with the continuous data acquisition.

In [ ]:
sq.io.convert_li_to_h5('your_logged_file.li', my_os='mac')

## Initalizing the Event Builder

The event building functionality assumes a specific organization of the data, which is as follows:

All continuous data to be triggered on is placed in one folder, in this case a folder called `continuous/`. The user should create a folder for the event building algorithm to save the triggered events (both randomly triggered and threshold triggered). In this example, we assume there is a folder called `trigger`.

Before acquiring any pulses, we initialize the `EventBuilder` class with these folders. During this, the user must specify how long (in bins) the saved traces should be. The user may optionally specify up to how many traces per dump should be saved before a new dump is created.

In [ ]:
nbins_trace = 10000 # traces will be 10000 bins long

EB = sq.daq.EventBuilder(
    './continuous/',
    './trigger/',
    nbins_trace,
    # maxevtsperdump=500, # default value to keep individual files relatively small
)

## Acquiring Random Triggers

The random triggering algorithm will look at all the total size of all of the continuous data files and randomly choose nonoverlapping sections from the files to save as "randoms". In this way, the randoms will encompass the entire dataset. The user must specify how many randoms should be saved, we'll acquire 500 randoms in this example.

In [ ]:
EB.acquire_randoms(500) # acquire 500 randoms

## Threshold Triggering

To trigger on some specified threshold, we use the so-called optimal filter formalism, which can quickly be described as a matched filter that takes into account the known noise environment. This is generally written in terms of the minimization of the below $\chi^2$
$$\chi^2 = \int_{-\infty}^\infty \mathop{\mathrm{d}f} \frac{|\tilde{v}(f) - A \mathrm{e}^{i \omega t} \tilde{s}(f)|^2}{J(f)},$$
where $\tilde{v}(f)$ is the Fourier transform of some signal with noise, $\tilde{s}(f)$ is some known template of the true signal (usually normalized to a height of unity), $A$ is the height estimate of the signal, $t$ is the start time of that signal, and $J(f)$ is the power spectral density (PSD) (i.e. the description of the noise environment in frequency space).

In this formalism, the expected baseline amplitude resolution $\sigma_A$ can be calculated, and we will set our eventual threshold based on the number of $\sigma_A$. Before doing that, let's define a template and a PSD to use for threshold triggering.

### Define template

Our template should be the expected shape of our signal. In this case, we have a trapezoidal pulse of width 1 ms and edge times of 50 us, so we will create that template, normalized to a height of unity.

In [ ]:
template = np.zeros(nbins_trace)
nbins_edge = int(edge_time * fs)
nbins_flat = int((pulse_width - edge_time) * fs) # note the pulse width is distance between the half way points of the edge times
template[
    len(template)//2:len(template)//2 + nbins_edge
] = np.arange(nbins_edge) / nbins_edge
template[
    len(template)//2 + nbins_edge:len(template)//2 + nbins_edge + nbins_flat
] = 1
template[
    len(template)//2 + nbins_edge + nbins_flat:len(template)//2 + 2 * nbins_edge + nbins_flat
] = np.arange(nbins_edge)[::-1] / nbins_edge

Note, sometimes the expected signal is not known a priori. In this case, one could instead average many of the measured signals, and then use that as a template (after normalizing to a height of unity).

### Calculate PSD

With our template defined, we next want to measure the PSD. To do this, we will open all of our randoms, remove those with pulses, and then use the remaining randoms to calculate the PSD. In these steps, we will use the SPLENDOR DSP package, `splendsp`, which can be installed via `pip`.

In [ ]:
FR = sq.io.Reader('./trigger/randoms_file.h5')
randoms = FR.get_data()

# IterCut is a class for iteratively marking events as "bad" based on some criteria
IC = sp.IterCut(randoms[:, 0], fs, plotall=False)
IC.pileupcut(template=template, cut=1.5)

f, psd = sp.calc_psd(randoms[IC.cmask, 0], fs=fs, folded_over=False)

# let's plot the folded over square root of PSD with and without the cuts, to show the difference
f_fold, psd_no_cuts_fold = sp.calc_psd(randoms[:, 0], fs=fs, folded_over=True)
f_fold, psd_fold = sp.calc_psd(randoms[IC.cmask, 0], fs=fs, folded_over=True)

fig, ax = plt.subplots()
ax.loglog(f_fold, psd_no_cuts_fold**0.5, color='r', label="Before Cuts")
ax.loglog(f_fold, psd_fold**0.5, color='b', label="After Cuts")
ax.set_ylabel("NEP [V/$\sqrt{\mathrm{Hz}}$]")
ax.set_xlabel("Frequency [Hz]")
ax.legend(edgecolor='k', framealpha=1)

### Run the Threshold Trigger

Now that we have our template and PSD, we can now run the threshold trigger. To do this, we use the `acquire_pulses` method, passing the template, psd, a turn-on (or activation) threshold (in units of number of $\sigma$), and which channel to trigger on. In this case, we will set an $20\sigma$ trigger on the channel we took data on (we only logged one channel, so this is the only option anyways).

Note that, when a section of the signal is near the threshold, then there may be fluctuations about the turn-on threshold that could be marked erroneously as separate events. To avoid this, we have a turn-off threshold that defaults to turn-on threshold minus 2, rather than the turn-on threshold, thus the filtered signal must drop below, in this example, $18\sigma$ to end the region of the data that is marked as single event.

The turn-off threshold may also be specified by the user. At low turn-on thresholds (below $5\sigma$), the turn-off threshold defaults to $3\sigma$. If the turn-on threshold is below $3\sigma$, then the turn-off threshold defaults to the turn-on threshold.

In [ ]:
EB.acquire_pulses(
    template,
    psd,
    20, # 20-sigma threshold
    0, # trigger on channel index 0
    threshold_off=None, # defaults to threshold_on - 2, i.e. threshold_off = 20 - 2 = 18
    mergewindow=None, # save all triggered events, set to a positive integer to merge events that are within this window
)

We have now acquired both random and threshold triggered events, which can then be read via `splendaq.io.Reader`.

In [ ]:
FR = sq.io.Reader()
triggered_pulses, metadata = FR.get_data('trigger/triggered_file.h5', include_metadata=True)

In [ ]:
fig, ax = plt.subplots()
ax.plot(triggered_pulses[0,0] - np.mean(triggered_pulses[0,0, :4000]), color='k', label="Triggered Data")
ax.plot(metadata['triggeramp'][0] * template, color='r', linestyle='dashed', label="Fit Template")
ax.set_ylabel("Amplitude [Arb.]")
ax.set_xlabel("Time [Bins]")
ax.set_xlim(4900, 5800)
ax.legend(edgecolor='k', framealpha=1)

And we can see that we have reconstructed the amplitude very well!